In [9]:
import pickle as pkl
import numpy as np
import scipy.spatial.distance as distance
import pandas as pd

# task_names = ['snli_ve', 'caption', 'refcoco', 'refcocoplus', 'refcocog', 'cola', 'mnli', 'sst2', 'rte', 'mrpc', 'qnli', 'qqp', 'food101', 'caltech101','dtd','eurosat','fgvc_aircraft','flowers102','oxfordpets','stanford_cars']
# task_names = ['mnli', 'sst2', 'rte', 'mrpc', 'qnli']
# task_names = ['vqa', 'snli_ve', 'caption', 'refcoco', 'refcocoplus', 'refcocog', 'cola', 'mnli', 'sst2', 'rte', 'mrpc', 'qnli', 'qqp', 'imagenet_preft_base', 'food101', 'caltech101','dtd','eurosat','fgvc_aircraft','flowers102','oxfordpets','stanford_cars']
# task_names = ['snli_ve', 'caption', 'refcoco', 'refcocoplus', 'refcocog']
task_names = [ 'refcoco', 'refcocoplus', 'refcocog']


task_embs = []

for task in task_names:
    with open(task+'/task_emb.pkl', 'rb') as fp:
        task_embs.append(pkl.load(fp))

In [10]:
def cosine(e0, e1):
    e0 = e0.reshape(-1)
    e1 = e1.reshape(-1)
    e0, e1 = e0/(e0 + e1 + 1e-6), e1/(e0 + e1 + 1e-6)
    return 1 - distance.cosine(e0, e1)

In [11]:
import warnings
warnings.filterwarnings('error')    
task_sim = np.zeros((len(task_embs), len(task_embs)))
for i, target_task in enumerate(task_embs):
    rank = []
    for k in target_task:
        if not 'adapter':
            continue
        if np.inf in target_task:
            continue
        cosine_sim = []
        flag = True
        
        for emb in task_embs[i+1:]:
            try:
                cosine_sim.append(cosine(target_task[k], emb[k]))
            except:
                flag=False
                break
        if flag:
            component_rank=sorted(enumerate(cosine_sim), key=lambda x:-x[1])
            rank.append(component_rank)
    task_sum = np.zeros(len(task_embs)-i-1)
    for r in rank:
        for tmp in r:
            task_sum[tmp[0]] += tmp[1]
    task_sum = task_sum / len(rank)
    task_sim[i, i+1:] = task_sum
task_sim

array([[0.        , 0.97240827, 0.96139766],
       [0.        , 0.        , 0.95890513],
       [0.        , 0.        , 0.        ]])

In [12]:
for i in range(len(task_sim)):
    for j in range(i):
        task_sim[i, j] = task_sim[j, i]
    task_sim[i, i] = 1 
task_sim = np.around(task_sim, decimals=4)
data = pd.DataFrame(task_sim, columns=task_names)
data.to_csv('task_sim.csv', index=None)